In [1]:
import pandas as pd
import numpy as np
import itertools

# 2. 시각화
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


# 3. 유틸
# tqdm 패키지는 반복문에 대해 얼마나 진척되었는지를 가시적으로 확인할 수 있도록 도와줍니다.
# https://github.com/tqdm/tqdm 사용법은 정말 간단합니다.
from tqdm.auto import tqdm


# 4. 설정
# 경고가 나와서, 출력이 많아지지 않기 위해 ignore를 설정해주었습니다.
import warnings
warnings.filterwarnings('ignore')


# 5. stats models
# 시계열 모델을 위한 ARIMA를 임포트 해주었습니다.
from statsmodels.tsa.arima_model import ARIMA
from pmdarima.arima import auto_arima

In [50]:
train_x = pd.read_csv("/train_x_df.csv")
train_y = pd.read_csv("/train_y_df.csv")

In [5]:
test_x = pd.read_csv("data/test_x_df.csv")

In [6]:
def buy_sell(signal, col1, col2):
    sigPriceBuy = []
    flag = -1

    for i in range(0, len(signal)):
        if signal[col1][i] > signal[col2][i] and flag != 1:
            sigPriceBuy.append(signal['close'][i])
            flag = 1

        elif signal[col1][i] < signal[col2][i] and flag != 0:
            sigPriceBuy.append(np.nan)
            flag = 0

        else:
            sigPriceBuy.append(np.nan)
    
    return (sigPriceBuy)

In [7]:
def make_obv(my_df):
    
    result = []

    for sample_id in my_df['sample_id'].unique().tolist():
        df = my_df[my_df['sample_id'] == sample_id]
        df.reset_index(inplace=True)
        OBV = []
        OBV.append(0)
        for i in range(1, len(df.close)):
            if df.close[i] > df.close[i-1]:
                OBV.append(OBV[-1] + df.volume[i])
            elif df.close[i] < df.close[i-1]:
                OBV.append(OBV[-1] - df.volume[i])
            else:
                OBV.append(OBV[-1])

        df['OBV'] = OBV
        df['OBV_EMA'] = df['OBV'].ewm(com=20).mean()

        buy_signal_price = buy_sell(df, 'OBV', 'OBV_EMA')
        df['Buy_Signal_Price'] = buy_signal_price

        result.append(df)                 

    output = pd.concat(result, axis=0)

    return output

In [8]:
test_x = make_obv(test_x)

In [9]:
TEST_SAMPLE_ID_LIST = test_x["sample_id"].unique().tolist()

In [10]:
def get_OBV(df,sample_id):    
    return df[df["sample_id"] == sample_id]['OBV'].values

In [12]:
# AIC 값이 최소인 p,d,q 값 직접 구하기
def model_fit(df, sample_id_list):
    result = []

    for sample_id in tqdm(sample_id_list):

        obv = get_OBV(df, sample_id)

        p = range(0, 5)
        d = range(0, 2)
        q = range(0, 5)

        pdq = list(itertools.product(p, d, q))

        aic = []
        for i in pdq:
            try:
                model = ARIMA(price_x, order=(i))
                model_fit = model.fit()
                aic.append(round(model_fit.aic, 2))
            except:
                continue

        optimal = [(pdq[i], j) for i, j in enumerate(aic) if j == min(aic)]
    
        # 2. ARIMA
        # 1) 모델 정의
        ARIMA_MODEL = {}
        ARIMA_MODEL_FIT = {}

        # 2) AR 모델 적용
        try:
            ARIMA_MODEL = ARIMA(obv, order = optimal[0][0])
            ARIMA_MODEL_FIT = ARIMA_MODEL.fit(trend = 'nc', full_output = True, disp = True)

        # 3) 수렴하지 않을 경우 p d q 를 1, 1, 0으로 사용
        except:
            ARIMA_MODEL = ARIMA(obv, order = (4,1,0))
            ARIMA_MODEL_FIT = ARIMA_MODEL.fit(trend = 'nc', full_output = True, disp = True)

        # 4) ARIMA 예측
        ARIMA_FORECAST  = ARIMA_MODEL_FIT.predict(1,120, typ='levels')

        # 3. 데이처 처리
        # 1) 최대 부분인 인덱스를 찾는데 해당 시점에 매도를 진행합니다.
        sell_time = np.argmax(ARIMA_FORECAST)

        # 2) 최대값을 찾습니다.
        max_val = np.max(ARIMA_FORECAST)

        obv_last_val = obv[1379]

        # 4. 투자 전략
        buy_quantity = 0

        # 1) typical_price가 1.1 이상이면 투자합니다.
        if max_val / obv_last_val > 2.5:
            buy_quantity = 1

        # 5. 결과
        result_list = [
                        sample_id,
                        buy_quantity,
                        sell_time,
                    ]

        result.append(result_list)

    return result

In [13]:
result = model_fit(test_x, TEST_SAMPLE_ID_LIST)

  0%|          | 0/760 [00:00<?, ?it/s]

C:\Users\pineb\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [14]:
submit_columns = [
                  "sample_id", 
                  "buy_quantity", 
                  "sell_time"
                  ]

submit = pd.DataFrame(data=result, columns=submit_columns)

In [15]:
#투자할 sample_id 갯수 확인
submit[submit["buy_quantity"] == 1].shape[0]  

21

In [16]:
def df2d_to_answer(df_2d):
    # valid_y_df로부터
    # open 가격 정보가 포함된
    # [샘플 수, 120분] 크기의 
    # 2차원 array를 반환하는 함수
    feature_size = df_2d.iloc[:,2:].shape[1]
    time_size = len(df_2d.time.value_counts())
    sample_size = len(df_2d.sample_id.value_counts())
    sample_index = df_2d.sample_id.value_counts().index
    array_2d = df_2d.open.values.reshape([sample_size, time_size])
    sample_index = list(sample_index)
    return array_2d, sample_index

In [18]:
FILE_NAME = "/0626_hong_obv.csv"

In [19]:
SUBMIT_PATH = "./data"
RESULT_PATH = SUBMIT_PATH + FILE_NAME

submit.to_csv(RESULT_PATH, index=False)